In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [3]:
input_folder = '../data/railways/denver_RTD/'
input_file = input_folder+'LightrailLines_Center.shp'

output_folder = '../data/output_images/DenverRTD/'
output_csv = output_folder+'DEN.csv'

update_folder = output_folder+'/set_3/'

### Import Routes

In [4]:
routes = gpd.read_file(input_file)

routes.head()

,OBJECTID,PROJECTCOR,SOURCE,Shape__Len,geometry
0,142,110,A-Line,87291.422422,(LINESTRING (-105.001090582883 39.753148821931...
1,143,112,G-Line,35177.373865,(LINESTRING (-105.137784196155 39.786794639271...
2,144,113,B-Line,7281.026112,(LINESTRING (-105.007403102142 39.806185928394...
3,145,109,L-Line,2816.281729,(LINESTRING (-104.988737522449 39.748187234728...
4,146,113,B-G Lines,20179.760544,(LINESTRING (-104.999945166902 39.754022112124...


In [5]:
# Number of routes
routes.shape

(84, 5)

In [20]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

18893 points


In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
#                     points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [22]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.tail()

,Name,Longitude,Latitude,Catenary
44,-105.130738422881_39.7236596419501,-105.130738,39.723660,1
45,-104.87303039361_39.5657739981097,-104.873030,39.565774,0
46,-104.97354011384_39.7584472286395,-104.973540,39.758447,0
47,-104.998248729006_39.7429942760075,-104.998249,39.742994,0
48,-105.001090582883_39.7531488219317,-105.001091,39.753149,0


In [27]:
'''
Get subselection of points excuding current points
'''
curr_points = list(zip(curr_df['Longitude'].tolist(),curr_df['Latitude'].tolist()))
# curr_points

route_points = []
for point in range(0,len(points),185):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

103


In [28]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]


df.head()

,Name,Longitude,Latitude
0,-105.001090582883_39.7531488219317,-105.001091,39.753149
1,-104.793074695928_39.7566259201284,-104.793075,39.756626
2,-104.933980528945_39.77629712982,-104.933981,39.776297
3,-104.957176173497_39.7740824619342,-104.957176,39.774082
4,-104.787524175553_39.7866594856642,-104.787524,39.786659


In [34]:
df.iloc[0]

Name         -105.001090582883_39.7531488219317
Longitude                              -105.001
Latitude                                39.7531
Name: 0, dtype: object

In [30]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [29]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [ ]:
'''
Get satellite preview for image
'''

row = 10

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

In [35]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 